# The dataset

### The dataset is called SQUAD.

### each datapoint is a question, a context that may contain the answer to the question, the start index of the answer, the answer


In [83]:
from datasets import load_dataset

dataset = load_dataset("squad")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


In [84]:
for q, a in zip(dataset["train"]["question"][:5], dataset["train"]["answers"]):
    print(f"{q} => {a} \n")

To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? => {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]} 

What is in front of the Notre Dame Main Building? => {'text': ['a copper statue of Christ'], 'answer_start': [188]} 

The Basilica of the Sacred heart at Notre Dame is beside to which structure? => {'text': ['the Main Building'], 'answer_start': [279]} 

What is the Grotto at Notre Dame? => {'text': ['a Marian place of prayer and reflection'], 'answer_start': [381]} 

What sits on top of the Main Building at Notre Dame? => {'text': ['a golden statue of the Virgin Mary'], 'answer_start': [92]} 



# this function adds the end of the answer to the dataset, and does some sanity checks


In [85]:
def compute_end_index(answers, contexts):
    """Add end index to answers"""
    # print(contexts)
    fixed_answers = []
    for answer, context in zip(answers, contexts):
        gold_text = answer["text"][0]
        # print(gold_text)
        answer["text"] = gold_text
        # print(answer["text"])

        start_idx = answer["answer_start"][0]
        answer["answer_start"] = start_idx

        # Make sure the starting index is valid and there is an answer
        assert start_idx >= 0 and len(gold_text.strip()) > 0
        end_idx = start_idx + len(gold_text)
        answer["answer_end"] = end_idx

        # Make sure the corresponding context matches the actual answer
        assert context[start_idx:end_idx] == gold_text

        fixed_answers.append(answer)
    return fixed_answers, contexts

In [86]:
for answer in dataset["train"]["answers"]:
    print(answer)
    break

{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


In [87]:
train_questions = dataset["train"]["question"]
print("Training data corrections")
train_answers, train_contexts = compute_end_index(
    dataset["train"]["answers"], dataset["train"]["context"]
)

test_questions = dataset["validation"]["question"]
print("\nValidation data correction")
test_answers, test_contexts = compute_end_index(
    dataset["validation"]["answers"], dataset["validation"]["context"]
)

Training data corrections

Validation data correction


# Implementing BERT


In [88]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained(
    "bert-base-uncased"
)  # using the pretrained tokenizer (there is no distinction between uppercase and lowercase)

OSError: bert-base-cased-squad2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

# Testing the tokenizer


In [48]:
context = "this is the context"
question = "this is the question"

"""
text - A single or batch of text sequences to be encoded by the tokenizer
text_pair - An optinal single or batcch of text sequences to be encoded by the tokenizer. useful for multipair(question and context)
"""

token_ids = tokenizer(
    text=context, text_pair=question, padding=False, return_tensors="tf"
)
print(token_ids)

{'input_ids': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=
array([[ 101, 2023, 2003, 1996, 6123,  102, 2023, 2003, 1996, 3160,  102]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


In [49]:
print(tokenizer.convert_ids_to_tokens(token_ids["input_ids"].numpy()[0]))

['[CLS]', 'this', 'is', 'the', 'context', '[SEP]', 'this', 'is', 'the', 'question', '[SEP]']


# Encoding the train and test dataset


In [50]:
# Encoding train
train_encodings = tokenizer(
    train_contexts, train_questions, truncation=True, padding=True, return_tensors="tf"
)

# Encoding test
test_encodings = tokenizer(
    test_contexts, test_questions, truncation=True, padding=True, return_tensors="tf"
)

print("train_encodings.shape: {}".format(train_encodings["input_ids"].shape))

train_encodings.shape: (87599, 512)


In [51]:
# convert the character-based indices to token-based indecis


def replace_char_with_token_indices(encodings, answers):
    """
    This function takes a set of BatchEncodings called encodings generated by the tokenizer and a set of
    answers( a list of dictionaries). Then it updates the provided encodings with two new keys:
    start_position and end_positions. These keys respectively hold the token-based indices denoting the
    start and end of the answer. if the asnwer is not found, set the start and end indices to the last token.
    To convert the existing character-based indeces to token-based indices, call char_to_token() provided by
    the BatchEncodings class.
    """

    start_positions = []
    end_positions = []
    n_updates = 0

    # Go through all the answers
    for i in range(len(answers)):
        # Get the token position for both start and end char positions
        start_positions.append(encodings.char_to_token(i, answers[i]["answer_start"]))
        end_positions.append(encodings.char_to_token(i, answers[i]["answer_end"] - 1))

        if start_positions[-1] is None or end_positions[-1] is None:
            n_updates += 1

        # if start position is None, the answer passage has been truncated
        # https://huggingface.co/transformers/custom_datasets.html#qa-squad

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length - 1

        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length - 1

    print("{}/{} had answers truncated".format(n_updates, len(answers)))
    encodings.update(
        {"start_positions": start_positions, "end_positions": end_positions}
    )

In [52]:
replace_char_with_token_indices(train_encodings, train_answers)
replace_char_with_token_indices(test_encodings, test_answers)

10/87599 had answers truncated
8/10570 had answers truncated


## Defining a Tensorflow dataset

data will
consist of two tuples: one containing inputs and the other containing the targets.

### The input tuple contains:

- Input token IDs – A batch of padded token IDs of size [batch size, sequence length]
- Attention mask – A batch of attention masks of size [batch size, sequence length]

### The output tuple contains:

- Start index of the answer – A batch of start indices of the answer
- End index of the answer – A batch of end indices of the answer


In [53]:
def data_gen(input_ids, attension_mask, start_positions, end_positions):
    """generator for data

    Since the data is already processed, it's a matter of reorganizing the already existing data to return
    """

    for inps, attn, start_pos, end_pos in zip(
        input_ids, attension_mask, start_positions, end_positions
    ):
        yield (inps, attn), (start_pos, end_pos)

In [54]:
from functools import partial
import tensorflow as tf

# Define the generator as a callable
train_data_gen = partial(
    data_gen,
    input_ids=train_encodings["input_ids"],
    attention_mask=train_encodings["attention_mask"],
    start_positions=train_encodings["start_positions"],
    end_positions=train_encodings["end_positions"],
)

# Define the dataset
train_dataset = tf.data.Dataset.from_generator(
    train_data_gen, output_types=(("int32", "int32"), ("int32", "int32"))
)

# shuffle the data
train_dataset = train_dataset.shuffle(1000)
print("\t Done")

	 Done


### split our dataset into two: a training set and a validation dataset. We will use the first

10,000 samples as the validation set. The rest of the data is used as the training set. Both datasets
will be batched using a batch size of 4


In [55]:
# Valid set is taken as the first 10000 samples in the shuffled set
valid_dataset = train_dataset.take(10000)
valid_dataset = valid_dataset.batch(4)


train_dataset = train_dataset.skip(10000)
train_dataset = train_dataset.batch(4)

### Creating Test dataset


In [62]:
print("Creating test data")

# Generator callable
test_data_gen = partial(
    data_gen,
    input_ids=test_encodings["input_ids"],
    attention_mask=test_encodings["attention_mask"],
    start_positions=test_encodings["start_positions"],
    end_positions=test_encodings["end_positions"],
)


test_dataset = tf.data.Dataset.from_generator(
    test_data_gen, output_types=(("int32", "int32"), ("int32", "int32"))
)

test_dataset = test_dataset.batch(8)

Creating test data


<_BatchDataset element_spec=((TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None)), (TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None)))>

In [57]:
from transformers import BertConfig, TFBertForQuestionAnswering

config = BertConfig.from_pretrained("bert-base-cased-squad2", return_dict=False)
print(config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "return_dict": false,
  "transformers_version": "4.45.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [58]:
model = TFBertForQuestionAnswering.from_pretrained("bert-base-uncased", config=config)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [90]:
def tf_wrap_model(model):
    """Wraps the huggingface's model with in the Keras Functional API"""
    # Define inputs
    input_ids = tf.keras.layers.Input(
        [
            None,
        ],
        dtype=tf.int32,
        name="input_ids",
    )
    attention_mask = tf.keras.layers.Input(
        [
            None,
        ],
        dtype=tf.int32,
        name="attention_mask",
    )
    # Define the output (TFQuestionAnsweringModelOutput)
    out = model([input_ids, attention_mask])

    # print(dir(out))
    # Get the correct attributes in the produced object to generate an
    # output tuple
    wrap_model = tf.keras.models.Model(
        [input_ids, attention_mask], outputs=(out.start_logits, out.end_logits)
    )
    return wrap_model

### Losses


In [91]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc = tf.keras.metrics.SparseCategoricalAccuracy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model_v2 = tf_wrap_model(model)
model_v2.compile(optimizer=optimizer, loss=loss, metrics=[acc])

AttributeError: 'tuple' object has no attribute 'start_logits'

In [92]:
model_v2.fit(train_dataset, validation_data=valid_dataset, epochs=3)